In [21]:
import pandas as pd
import numpy as np
import gzip
import json
import sys
from langdetect import detect, LangDetectException
import nltk
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.decomposition import PCA
from scipy.stats import zscore
from pyod.models.mad import MAD

sys.path.append("../utils/")
from myFunctions import jsonGzipToDataframe2, toDommyColumns, joinLanguage

Descargamos el paquete de palabras que utilizara el analizador se sentimientos

In [25]:
nltk.download('vader_lexicon') # ¡SE DEBE EJECUTAR SOLO UNA VEZ!

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/rafael/nltk_data...


True

Cargamos dentro de dataframes Pandas los CSVs comprimido 

In [3]:
df = pd.read_csv('../datasource/user_reviews_chewed.csv.gz')

In [4]:
dfSteam = pd.read_csv('../datasource/steam_games_chewed.csv.gz')

Nos quedaremos con las columas que necesitaremos agregar a df

In [5]:
dfSteam = dfSteam[['id', 'developer']]

Quitamos los 'developer' que tengan valor NaN 

In [6]:
dfSteam = dfSteam.dropna(subset = ['developer']).reset_index(drop = True)

In [ ]:
dfSteam

In [ ]:
df.info()

Quitamos los reviews que tengan valor NaN 

In [7]:
df = df.dropna(subset = ['review']).reset_index(drop = True)

Y nos quedamos con las columnas que se van a utilizar en el analisis de sentimiento

In [8]:
df = df[['item_id', 'posted', 'recommend', 'review']]

Ajustamos rapidamente el tipo de dato de item_id

In [9]:
df['item_id'] = df['item_id'].astype('int64')  # Convierte a entero de 64 bits

In [ ]:
df.head()

In [ ]:
df['recommend'].value_counts()

#### Save 💾

En este punto el dataframe df esta listo para ser la fuente de datos de la función 'userdata()' que respondera desde la<br>API. Siendo asi, tomaremos una instantanea del estado actual de los datos y los persistiremos en un CSV. 

In [10]:
# Guardar el DataFrame en un archivo .csv comprimido con gzip
df.to_csv('../datasource/user_reviews_Api.csv.gz', compression='gzip', index = False)

Agreguemos a df la columna 'developer' que esta en dfSteam

In [72]:
# Let's add the 'gamesGenre' columns to 'dfUserItems'. Now we have the years of the games
# next to the items!
dfUnion = df.merge(dfSteam, left_on = 'item_id', right_on = 'id', how = 'inner')

Obtengamos el año del comentario del contenido de la columna 'posted' y guardemoslo en una nueva columna 'year'

In [73]:
# Verifiquemos el formato de fecha valido de la columna 'posted'
# Usando la exresion regular 'Posted |,|\.' podemos verificar la forma en la que esta escrita la 
# fecha. Estamos buscando un texto como por ejemplo:
#              Posted xxx, yyyy.
# En donde yyyy sera el año
dfUnion['posted'] = pd.to_datetime(
    dfUnion['posted'].astype(str).str.replace(r'Posted |,|\.', '', regex = True), errors = 'coerce')

# Crea la columna 'year' a partir de 'posted'
dfUnion['year'] = dfUnion['posted'].dt.year.astype('Int64')

# La columna posted ya no hace falta y aprovecho para quitar 'id'
dfUnion.drop(['posted', 'id'], axis = 1, inplace = True)

Quitamos los años que tengan valor NaN 

In [74]:
dfUnion = dfUnion.dropna(subset = ['year']).reset_index(drop = True)

In [ ]:
dfUnion

#### Idioma 🇬🇧

In [94]:
dfUnionLanguage = dfUnion.copy()

In [95]:
#print('dfUnionLanguage:', id(dfUnionLanguage))
print('dfUnion', id(dfUnion))
print('dfUnionLanguage:', id(dfUnionLanguage))

dfUnion 139818262572208
dfUnionLanguage: 139818263587488


Agreguemos la columna 'language' que corresponde al idioma del comentario del usuario

In [96]:
dfUnionLanguage = joinLanguage(dfUnionLanguage, 'review')


De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1.

In [97]:
print('Reseñas nulas:',dfUnionLanguage['language'].isnull().sum())
dfUnionLanguage['language'].fillna(1, inplace=True)
print('Reseñas nulas despues de la conversion:',dfUnionLanguage['language'].isnull().sum())

Reseñas nulas: 391
Reseñas nulas: 0


¿Cual es el idioma que mas se repite?

In [98]:
dfUnionLanguage['language'].value_counts().head()

language
en    39408
pt     1524
es      909
de      824
so      693
Name: count, dtype: int64

Nos quedaremos con 'en'

In [99]:
# Borramos los comentarios que no estan en ingles y los que tienen valor NaN
dfUnionLanguage = dfUnionLanguage.loc[
      (dfUnionLanguage['language'] == 'en')
      #(dfUnionLanguage['language'].notna()) & (dfUnionLanguage['language'] == 'en')
    ].reset_index(drop = True)
# Todo quedo en ingles asi que es redundante la columna 'language'
dfUnionLanguage.drop('language', axis = 1, inplace = True)

In [100]:
dfUnionLanguage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39408 entries, 0 to 39407
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item_id    39408 non-null  int64 
 1   recommend  39408 non-null  object
 2   review     39408 non-null  object
 3   developer  39408 non-null  object
 4   year       39408 non-null  Int64 
dtypes: Int64(1), int64(1), object(3)
memory usage: 1.5+ MB


Proceso de limpieza de la columna 'review'

In [101]:
# Cambiemos todo a minusculas para evitar ambiguedades en las palabras.
dfUnionLanguage.loc[:, 'review'] = dfUnionLanguage['review'].str.lower()

# Fuera los caracteres especiales y signos de puntuacion.
dfUnionLanguage['review'] = dfUnionLanguage['review'].replace('[^A-Za-z0-9\s]+', '', regex = True)
dfUnionLanguage.loc[:, 'review'] = dfUnionLanguage['review'].str.replace('[^\w\s]', '', regex = True)

#### Analisis de sentimiento 🫶

In [103]:
def sentimentAnalysis(df):
    analyzer = SentimentIntensityAnalyzer()

    # Calcular las puntuaciones de sentimiento a cada texto en la columna 'review'
    # Creamos la columna 'compoundScore' para guardad el calculo
    df['compoundScore'] = df['review'].apply(
        lambda review: analyzer.polarity_scores(review)['compound'])
    # Reflejamos en la nueva columna 'sentimentAnalysis' la escala '0' si es malo,
    # '1' si es neutral y '2' si es positivo
    df['sentimentAnalysis'] = df['compoundScore'].apply(
        lambda score: 0 if score < 0 else (1 if score == 0 else 2))

    # Conteo de reviews por score
    scoreCounts = df['sentimentAnalysis'].value_counts()

    # Conteo de reviews en blanco
    blankReviewsCount = df['review'].isnull().sum()

    # Total de reviews
    totalReviews = len(df)

    dataset = {
      # Conteo de reviews por score
      'scoreCounts': scoreCounts,

      # Conteo de reviews en blanco
      'blankReviewsCount': blankReviewsCount,

      # Calcular porcentajes
      'scorePercentages': (scoreCounts / totalReviews * 100).round(2),
      'blankReviewsPercentage': (blankReviewsCount / totalReviews * 100).round(2)
    }
     
    # Ya no necesitaremos las columnas 'review' y 'compoundScore'
    df.drop(['review','compoundScore'], axis = 1, inplace = True)  

    #return df, dataset

In [105]:
# Ejecutamos el analisis de sentimiento y lo incorporamos como una nueva columna al dataframe
sentimentAnalysis(dfUnionLanguage)
#dfUserReviews, d = sentimentAnalysis(dfUnionLanguage)

In [106]:
dfUnionLanguage

,item_id,recommend,developer,year,sentimentAnalysis
0,1250,True,Tripwire Interactive,2011,2
1,1250,True,Tripwire Interactive,2015,2
2,1250,True,Tripwire Interactive,2013,0
3,1250,True,Tripwire Interactive,2010,2
4,1250,True,Tripwire Interactive,2015,0
...,...,...,...,...,...
39403,367780,True,"Soloweb Studios,Ravens Eye Studio",2015,2
39404,367780,True,"Soloweb Studios,Ravens Eye Studio",2015,2
39405,305920,False,ShaunJS,2015,0
39406,306040,True,"Antanas Marcelionis,Renė Petrulienė",2015,2


In [107]:
dfUnionLanguage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39408 entries, 0 to 39407
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   item_id            39408 non-null  int64 
 1   recommend          39408 non-null  object
 2   developer          39408 non-null  object
 3   year               39408 non-null  Int64 
 4   sentimentAnalysis  39408 non-null  int64 
dtypes: Int64(1), int64(2), object(2)
memory usage: 1.5+ MB


#### Save 💾

En este punto el dataframe df esta listo para ser la fuente de datos de las funciones 'best_developer_year' y<br>'developer_reviews_analysis()' que responderan desde la API. Siendo asi, tomaremos una instantanea del estado actual<br>de los datos y los persistiremos en un CSV. 

In [108]:
# Guardar el DataFrame en un archivo .csv comprimido con gzip
df.to_csv('../datasource/user_reviews_2Api.csv.gz', compression='gzip', index = False)

#### LA CONSULTA PARA LA API

In [137]:
def best_developer_year(year: int) -> pd.DataFrame:
  """
  Devuelve el top 3 de desarrolladores con juegos mas recomendados por usuarios para el año dado.
  Params:
    year (int): El año a considerar.
  Returns:
    pd.DataFrame: Un DataFrame con las columnas 'developer' y 'recomendaciones', ordenado por
    'recomendaciones' de forma descendente y limitado a las 3 primeras filas.
  """
  if not isinstance(year, int):
    return print(f'{year} fuera de rango')
  # Filtrar los juegos recomendados para el año dado
  juegosRecomendados = dfUnionLanguage[
      (dfUnionLanguage['year'] == year) &
      (dfUnionLanguage['recommend'] == True) &
      (dfUnionLanguage['sentimentAnalysis'] == 2)
  ]  
  
  # Contar las recomendaciones por desarrollador
  recomendacionesXDesarrollador = (
      juegosRecomendados.groupby('developer')['item_id'].nunique()
  )  
  
  # Convertir a DataFrame y ordenar por recomendaciones de forma descendente
  topDesarrolladores = recomendacionesXDesarrollador.to_frame(name = 'recomendaciones').reset_index()
  topDesarrolladores = topDesarrolladores.sort_values(by = 'recomendaciones', ascending = False)  

  topDesarrolladoresLista = []
  for i in range(0, 3):
      puesto = f"Puesto {i + 1}"
      try:
        desarrollador = topDesarrolladores.loc[i, 'developer']
      except Exception as e:
         return print(f'{year} fuera de rango\n{e}')
      topDesarrolladoresLista.append({puesto: desarrollador})
  return topDesarrolladoresLista

In [144]:
best_developer_year(2015)

[{'Puesto 1': '10th Art Studio,Adventure Productions'},
 {'Puesto 2': '11 bit studios'},
 {'Puesto 3': '14° East'}]

In [146]:
import pandas as pd

def developer_reviews_analysis(developer: str) -> dict:
    """Devuelve un diccionario con el nombre del desarrollador como llave y una lista con la
    cantidad de reseñas negativas y positivas del desarrollador.
    Params:
        developer (str): El nombre del desarrollador.
    Returns:
        dict: Un diccionario con la siguiente estructura: {'nombre_desarrollador': ['Negative = X', 'Positive = Y']}
    """
    # Filtrar las reseñas del desarrollador especificado
    developer_reviews = dfUnionLanguage[dfUnionLanguage['developer'] == developer]

    # Contar las reseñas positivas y negativas
    negative_count = developer_reviews[developer_reviews['sentimentAnalysis'] == 0].shape[0]
    positive_count = developer_reviews[developer_reviews['sentimentAnalysis'] == 2].shape[0]

    # Crear el diccionario de resultados
    resultado = {
        developer: [f"Negative = {negative_count}", f"Positive = {positive_count}"]
    }

    return resultado


In [147]:
developer_reviews_analysis('Tripwire Interactive')

{'Tripwire Interactive': ['Negative = 83', 'Positive = 243']}

In [ ]:
# Convierte a entero de 64 bits
df['item_id'] = df['item_id'].astype('int64')